In [1]:
import pandas as pd
import convertePlanilha

In [2]:
# Faz leitura do arquivo excel a partir de um caminho relativo
df = pd.read_excel(r'dataset\\2024-05-09-emeronweb.xls')

df_clean = convertePlanilha.clean_data(df.copy())

In [ ]:
df_clean

In [ ]:
df_clean.columns

In [ ]:
df_clean.info()

- Cursos por público

In [ ]:
df_clean['Público'].value_counts()

- Crio um dataframe para trabalhar especificamente as métricas

In [ ]:
df_metricas = df_clean[['Início','Atividade','Vagas','N. Alunos','Concluintes','Não Concluintes','CH']]

df_metricas.head(3)

In [ ]:
df_metricas.drop(columns='CH').value_counts()

## Numero de vagas, alunos, concluintes, carga horária

### Métricas:
- Alunos por vagas (total anual)
- Concluintes por não concluintes (total anual)
- Vagas abertas por alunos inscritos no mês
- Concluintes por não concluintes (por mês de encerramento do curso)
- Total de atividades por mês

In [ ]:
df_vagas_alunos_concluintes = df_metricas.drop(columns=['Início', 'Atividade','CH', 'Não Concluintes']).sum()
df_vagas_alunos_concluintes

- Carga horária total

In [ ]:
df_metricas['CH'].sum()

- Total de Atividaes

In [ ]:
df_metricas['Atividade'].count()

- Total de atividades por mês

In [ ]:
df_clean.groupby("mes_inicio")["Atividade"].count()

- Total de vagas abertas por mês

In [ ]:
df_clean.groupby('mes_inicio')['Vagas'].sum()

- Mes que mais encerrou cursos por concluientes

In [ ]:
df_clean.groupby('mes_fim')['Concluintes'].sum()

In [ ]:
### Filtro Aguardando: Vagas, Alunos, Concluintes e Não Concluintes, exceto mês 12
df_filtroAguardando = df_clean[['Situação','mes_inicio','mes_fim','Vagas','N. Alunos','Concluintes','Não Concluintes']]

df_filtroAguardando = df_filtroAguardando[(df_clean['Situação'] == 'Aguardando') & (df_clean['mes_fim'] < 12)]

df_filtroAguardando.sample(3)

In [ ]:
bimestre = df_clean[['Início','Vagas']].groupby(pd.Grouper(key='Início', freq='2MS')).sum()
bimestre

In [ ]:
trimestre = df_clean[['Início','Vagas']].groupby(pd.Grouper(key='Início', freq='3MS')).sum()
trimestre

In [ ]:
quadrimestre = df_clean[['Início','Vagas']].groupby(pd.Grouper(key='Início', freq='4MS')).sum()
quadrimestre

In [ ]:
semestre = df_clean[['Início','Vagas']].groupby(pd.Grouper(key='Início', freq='6MS')).sum()
semestre

# Gráficos

- Número cursos executados por número de cursos aguardando

In [ ]:
df_clean.columns

In [45]:
df_referencia = df_clean[['N. Alunos','mes_referencia']].groupby('mes_referencia')['N. Alunos'].sum().to_frame().reset_index()
df_referencia

,mes_referencia,N. Alunos
0,01 - janeiro,109
1,02 - fevereiro,541
2,03 - março,666
3,04 - abril,1601
4,05 - maio,173
5,13 - Anual,3407


In [46]:
px.bar(df_referencia,x='mes_referencia',y='N. Alunos').show()

In [ ]:
df_clean['Situação'].value_counts().reset_index()

# Configurando os meses do ano e dias da semana

In [ ]:
dias_da_semana = ('Domingo','Segunda-feira','Terça-feira','Quarta-feira','Quinta-feira','Sexta-feria','Sábado')


In [ ]:
dias_da_semana[1]

In [ ]:
df_clean['dia_inicio'].sample(3)

In [ ]:
df_clean['Início'].sample(3)

In [ ]:
def map_numero_to_dia_da_semana(numero):
    if numero < 1 or numero > 6:
        raise ValueError("O número deve estar entre 1 e 7")
    return dias_da_semana[numero -1]

In [ ]:
df_clean['dia_da_semana_nome_inicio'] = df_clean['Início'].dt.strftime('%A')
df_clean['dia_da_semana_nome_fim'] = df_clean['Fim'].dt.strftime('%A')
df_clean['dia_da_semana_nome'].sample(5)

In [ ]:
meses = ('Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho', 'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro')

1 - Essa primeira maneira eu crio uma função que receba um número e retorne o valor correspondente na tupla, este valor deverá ser adicionado na nova coluna

In [ ]:
def map_numero_to_mes(numero):
    if numero < 1 or numero > 12:
        raise ValueError("O número deve estar entre 1 e 12.")
    return meses[numero - 1]

In [ ]:
df_clean.columns

Explicação:

A função map_numero_to_mes recebe um número de mês como argumento e retorna o nome do mês correspondente.
A função apply aplica a função map_numero_to_mes a cada elemento da coluna numero_meses e armazena o resultado na nova coluna meses.
O DataFrame agora possui a coluna meses preenchida com os nomes dos meses completos, correspondentes aos numerais na coluna numero_meses.
Observações:

Este código assume que a tupla meses contém os nomes dos meses na ordem correta (de Janeiro a Dezembro).
Se a ordem dos nomes dos meses na tupla for diferente, você precisará ajustar a função map_numero_to_mes de acordo.
Certifique-se de que os valores na coluna numero_meses sejam números inteiros entre 1 e 12.

> Gemini

In [ ]:
df_clean['meses_inicio_nominais'] = df_clean['mes_inicio'].apply(map_numero_to_mes)
df_clean['meses_inicio_nominais'].sample(3)

2 - Dessa maneira posso iterar item por item para adicionar um valor a tabela

3 - Essa maneira utiliza o a função dt.strftime da biblioteca datetime, como resultado o nome dos meses retornam com letra minúscula

> link  [documentação da biblioteca strftime](https://strftime.org/)

In [ ]:
df_clean['meses_inicio_nomeados'] = df_clean['Início'].dt.strftime('%B')
df_clean['meses_fim_nomeados'] = df_clean['Fim'].dt.strftime('%B')
df_clean['meses_inicio_nomeados'].sample(3)

In [ ]:
resumo = df_clean.get(['Início','dia_da_semana_nome','meses_inicio_nomeados','dia_da_semana_nome_fim','meses_fim_nomeados'])
resumo

In [ ]:
df_sem_autoinstrucional = df_clean[(df_clean['mes_inicio'] != 2) & (df_clean['mes_fim'] != 12)]

In [ ]:
df_sem_autoinstrucional